<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Emotion_Detection_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d msambare/fer2013
!unzip \*.zip && rm *.zip

In [ ]:
tf.keras.preprocessing.image.load_img('')